In [1]:
""" First attempt to build a ConvNet in Keras
	for CV (Computer Vision), more specifically for 
	Image Recognition (classification problem).
"""

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K 
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal  

# For reproducibility
np.random.seed(42)

Using TensorFlow backend.


In [2]:
batch_size    = 128
epochs        = 200
iterations    = 391
num_classes   = 10
dropout       = 0.5
log_filepath  = './nin_bn'

In [3]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

    return x_train, x_test

In [4]:
def scheduler(epoch):
    if epoch <= 60:
        return 0.05
    if epoch <= 120:
        return 0.01
    if epoch <= 160:    
        return 0.002
    return 0.0004

In [5]:
def build_model(act):
    model = Sequential()

    model.add(Conv2D(192, (5, 5), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001), activation=act, input_shape=x_train.shape[1:]))
    model.add(Conv2D(160, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001), activation=act))
    model.add(Conv2D(96, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))

    model.add(Dropout(dropout))

    model.add(Conv2D(192, (5, 5), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001), activation=act))
    model.add(Conv2D(192, (1, 1),padding='same', kernel_regularizer=keras.regularizers.l2(0.0001), activation=act))
    model.add(Conv2D(192, (1, 1),padding='same', kernel_regularizer=keras.regularizers.l2(0.0001)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))

    model.add(Dropout(dropout))

    model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001), activation=act))
    model.add(Conv2D(192, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001), activation=act))
    model.add(Conv2D(10, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(0.0001)))
    model.add(BatchNormalization())
    model.add(Activation(act))

    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))

    sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

In [6]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train, x_test = color_preprocessing(x_train, x_test)

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
# print('Using real-time data augmentation.')
# datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.125,height_shift_range=0.125,fill_mode='constant',cval=0.)
# datagen.fit(x_train)

#     # start training
#     model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),steps_per_epoch=iterations,epochs=epochs,callbacks=cbks,validation_data=(x_test, y_test))
#     model.save('nin_bn.h5')

In [7]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [8]:
activations = [e_swish_1, e_swish_2, swish, "relu", "sigmoid", "tanh"]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_1', 'e_swish_2', 'swish', 'relu', 'sigmoid', 'tanh']


In [9]:
act, act_name = activations[1], names[1]
model = build_model(act)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 192)       14592     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 160)       30880     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 96)        15456     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 96)        384       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 96)        0         
__________

In [ ]:
his = model.fit(x_train, y_train, epochs=epochs, callbacks=cbks, validation_data=(x_test, y_test), verbose=1)
model.save('nin_bn_e_swish_2.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 934s - loss: 1.5441 - acc: 0.4869 - val_loss: 1.4116 - val_acc: 0.5342
Epoch 2/200
50000/50000 [==============================] - 622s - loss: 1.2377 - acc: 0.6118 - val_loss: 1.3852 - val_acc: 0.5653
Epoch 3/200
50000/50000 [==============================] - 623s - loss: 1.1267 - acc: 0.6575 - val_loss: 1.1117 - val_acc: 0.6649
Epoch 4/200
50000/50000 [==============================] - 615s - loss: 1.0516 - acc: 0.6904 - val_loss: 1.0278 - val_acc: 0.7000
Epoch 5/200
50000/50000 [==============================] - 613s - loss: 0.9840 - acc: 0.7212 - val_loss: 1.0472 - val_acc: 0.7010
Epoch 6/200
50000/50000 [==============================] - 619s - loss: 0.9457 - acc: 0.7438 - val_loss: 0.9088 - val_acc: 0.7573
Epoch 7/200
50000/50000 [==============================] - 619s - loss: 0.9207 - acc: 0.7579 - val_loss: 0.9153 - val_acc: 0.7613
Epoch 8/200
50000/50000 [===============

In [ ]:
keras.backend.get_session().close()
keras.backend.set_session(keras.backend.tf.Session())